In [1]:
import pandas as pd
import numpy as np

In [2]:
#mount Google drive
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [3]:
zika_url = '/gdrive/My Drive/Zika-Github/cdc_zika.csv'

In [28]:
df = pd.read_csv(zika_url)
df.head(10)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,report_date,location,location_type,data_field,data_field_code,time_period,time_period_type,value,unit
0,2016-03-19,Argentina-Buenos_Aires,province,cumulative_confirmed_local_cases,AR0001,NaN,NaN,0,cases
1,2016-03-19,Argentina-Buenos_Aires,province,cumulative_probable_local_cases,AR0002,NaN,NaN,0,cases
2,2016-03-19,Argentina-Buenos_Aires,province,cumulative_confirmed_imported_cases,AR0003,NaN,NaN,2,cases
3,2016-03-19,Argentina-Buenos_Aires,province,cumulative_probable_imported_cases,AR0004,NaN,NaN,1,cases
4,2016-03-19,Argentina-Buenos_Aires,province,cumulative_cases_under_study,AR0005,NaN,NaN,127,cases
5,2016-03-19,Argentina-Buenos_Aires,province,cumulative_cases_discarded,AR0006,NaN,NaN,0,cases
6,2016-03-19,Argentina-CABA,province,cumulative_confirmed_local_cases,AR0001,NaN,NaN,0,cases
7,2016-03-19,Argentina-CABA,province,cumulative_probable_local_cases,AR0002,NaN,NaN,0,cases
8,2016-03-19,Argentina-CABA,province,cumulative_confirmed_imported_cases,AR0003,NaN,NaN,9,cases
9,2016-03-19,Argentina-CABA,province,cumulative_probable_imported_cases,AR0004,NaN,NaN,0,cases


In [29]:
df.drop('time_period', inplace=True, axis=1)
df.drop('time_period_type', inplace=True, axis=1)

In [30]:
df.head()

,report_date,location,location_type,data_field,data_field_code,value,unit
0,2016-03-19,Argentina-Buenos_Aires,province,cumulative_confirmed_local_cases,AR0001,0,cases
1,2016-03-19,Argentina-Buenos_Aires,province,cumulative_probable_local_cases,AR0002,0,cases
2,2016-03-19,Argentina-Buenos_Aires,province,cumulative_confirmed_imported_cases,AR0003,2,cases
3,2016-03-19,Argentina-Buenos_Aires,province,cumulative_probable_imported_cases,AR0004,1,cases
4,2016-03-19,Argentina-Buenos_Aires,province,cumulative_cases_under_study,AR0005,127,cases


In [31]:
df.tail()

,report_date,location,location_type,data_field,data_field_code,value,unit
107614,2016-06-28,United_States_Virgin_Islands,territory,confirmed_conjunctivitis,VI0017,7,cases
107615,2016-06-28,United_States_Virgin_Islands,territory,confirmed_eyepain,VI0018,13,cases
107616,2016-06-28,United_States_Virgin_Islands,territory,confirmed_headache,VI0019,14,cases
107617,2016-06-28,United_States_Virgin_Islands,territory,confirmed_malaise,VI0020,5,cases
107618,2016-06-28,United_States_Virgin_Islands,territory,zika_no_specimen,VI0021,2,cases


In [32]:
df.dtypes

report_date        object
location           object
location_type      object
data_field         object
data_field_code    object
value              object
unit               object
dtype: object

In [33]:
df.dropna(inplace=True)

In [35]:
df['value']=pd.to_numeric(df['value'], errors='coerce')
df.dropna(inplace=True)

In [41]:
df = df.astype({"value":int})

In [42]:
# Get names of indexes for which column value has value 0
indexNames = df[ df['value'] == 0 ].index
# # Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)

In [44]:
df.tail()

,report_date,location,location_type,data_field,data_field_code,value,unit
107614,2016-06-28,United_States_Virgin_Islands,territory,confirmed_conjunctivitis,VI0017,7,cases
107615,2016-06-28,United_States_Virgin_Islands,territory,confirmed_eyepain,VI0018,13,cases
107616,2016-06-28,United_States_Virgin_Islands,territory,confirmed_headache,VI0019,14,cases
107617,2016-06-28,United_States_Virgin_Islands,territory,confirmed_malaise,VI0020,5,cases
107618,2016-06-28,United_States_Virgin_Islands,territory,zika_no_specimen,VI0021,2,cases


In [45]:
df.dtypes

report_date        object
location           object
location_type      object
data_field         object
data_field_code    object
value               int64
unit               object
dtype: object

In [48]:
# usar RE para reemplazar todas las fechas al mismo formato